In [13]:
import os
import torch

In [14]:


ssd_checkpoint_file = "/home/andrew/Development/WIT-Asset-Detection/ssd_logs/2023_03_01__13_42_11/andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.pt"
output_file_name = os.path.splitext(os.path.basename(ssd_checkpoint_file))[0] + ".torchscript"
print(f"{output_file_name=}")

output_file_name='andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.torchscript'


In [15]:
checkpoint = torch.load(ssd_checkpoint_file)
start_epoch = checkpoint['epoch'] + 1
print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)


from ssd_model import SSD300
from ssd_utils import label_map
n_classes = len(label_map)
model = SSD300(n_classes=n_classes)

model.load_state_dict(checkpoint['model'])
model.eval()


Loaded checkpoint from epoch 551.


Loaded base model.



SSD300(
  (base): VGGBase(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), 

In [16]:
print(checkpoint.keys())

dict_keys(['epoch', 'model', 'optimizer'])


In [19]:
SSD_c, SSD_h, SSD_w = 3, 300, 300
sample_image_input = torch.rand(1, SSD_c, SSD_h, SSD_w)

In [20]:
traced_script_module = torch.jit.trace(model, sample_image_input)

Check the output make sure it's the same

In [24]:
traced_output = traced_script_module(sample_image_input)

In [25]:
actual_output = model(sample_image_input)

In [26]:
print("Norm=", torch.norm(traced_output[0] - actual_output[0]))

Norm= tensor(0., grad_fn=<NormBackward1>)


In [23]:
traced_script_module.save(output_file_name)
print("Done! Output to:", output_file_name)

Done! Output to: andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.torchscript
